In [ ]:
# !wget https://huggingface.co/datasets/mesolitica/mixtral-magicoder/resolve/main/data/python-00000-of-00001.jsonl
# !wget https://huggingface.co/datasets/mesolitica/mixtral-magicoder/resolve/main/data/pythonanalytics-00000-of-00001.jsonl

In [ ]:
# !wget https://huggingface.co/datasets/m-a-p/Code-Feedback/resolve/main/Code-Feedback.jsonl
# !wget https://huggingface.co/datasets/teknium/OpenHermes-2.5/resolve/main/openhermes2_5.json

In [ ]:
import json

filtered = []
with open('Code-Feedback.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if '```python' in l['messages'][1]['content']:
            filtered.append(l)

In [ ]:
len(filtered)

In [ ]:
!mkdir code-feedback

In [ ]:
from agent import ClientJupyterKernel, Agent, Generator
import os

def answer(q, i):
    filename = f'code-feedback/{i}.json'
    if os.path.exists(filename):
        return
    
    jupyter_kernel_url = 'http://localhost:8081/execute'
    openai_base_url = 'http://localhost:8005/v1'
    model_name = 'xingyaoww/CodeActAgent-Mistral-7b-v0.1'

    generator = Generator(model_name, openai_base_url)
    prompt = q['messages'][0]['content']
    
    r = None
    agent = None
    for k in range(2):
        try:
            CONV_ID = f'code-feedback-{i}-{k}'
            code_executor = ClientJupyterKernel(jupyter_kernel_url, CONV_ID)
            agent = Agent(generator, code_executor, conv_id=CONV_ID, silent = True)
            r = agent.run_prompt(prompt, n_max_executions = 10)
            agent.code_executor.shutdown()
            if '<execute>' in r[2]['content']:
                break
        except Exception as e:
            print(e)
            if agent is not None:
                agent.code_executor.shutdown()
            pass
    with open(filename, 'w') as fopen:
        json.dump({'original': q, 'result': r}, fopen)

In [ ]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [ ]:
urls = [(q, no) for no, q in enumerate(filtered)]

In [1]:
!docker rm $(docker ps -a -q --filter="name=conv-code-feedback*") -f

faca0f92897f
7c4f7131bc81
80dc2731c940
903e1ea55487
5cd53641194a
4cbf14fc1249
9172607366b1
746153eed34c
5aa6cc423cc5
7376802b494f
b151d96e88e6
00f490a2ffab
8125c237def7
e7c844b8d513
ba67b4dedaf8
2b9759fbe484
8db02ee12a89
8a7be2aa6447
8fe9a98b299a
01a5bf3b8980
7a58d5a81908
3bcaf523c405
d900e9de2490
55ecc18fcb0a
a71c71beb9d7
c0fd2377f706
99fa3b988bff
3638bf4135dc
aaf6692ea87a
faca22476acc
02279d62a55e
6c106115db05
cf3ab8fed1be
90b9f2a34f5e
4435f905c213
f90d0f8a94dc
99cdafa4efa7
a0cbec9391ad
8f856c563641
888e895858d3
44186ebb4b63
e09f920a7304
30b582db8365
a7a40a711691
366cbd2c13e8
feee3aedb3c8
3cdb8b98e44e
1426d0cbaa84
024c9b463127
0d8f13589954
e5b4ac517bf5
1a46eead8a8e
dfeb52a620f2
05b58422d128
08004c42befb
ba3b6f9f00f2
a33255844f8b
2018db66102d
fa8ed6f9e40b
c0075f5152b2
0fbef30c9704
f23c898d57e0
27178a5ebae1
13e86e4fbcbc
327dde3e0aac
ece3d0eae855
73541548d5c8
65beea7f7f3a
4a012f592388
710ce7cb8245
33b13db52b45
158e2079b2f5
ccdfbb769596
95f628c70147
04321f8112b9
3f726fbbc894
4d13dbcafef5

In [ ]:
# !rm code-feedback/210.json
answer(*urls[2])

In [ ]:
# !rm code-feedback/*.json

In [ ]:
# with open('code-feedback/78.json') as fopen:
#     for r in json.load(fopen)['result']:
#         print(r)

In [ ]:
from threading import Thread
from queue import Queue
from tqdm import tqdm

queue = Queue()
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [ ]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()